# TYTX - JSON Serialization

This notebook shows how TYTX handles JSON serialization, allowing type preservation during data exchange.

In [ ]:
from decimal import Decimal
from datetime import date, datetime, timezone
import json

from genro_tytx import as_json, as_typed_json, from_json

## The Problem: JSON Loses Types

Standard JSON only supports: string, number, boolean, null, array, object.
Types like `Decimal`, `date`, `datetime` are lost.

In [ ]:
# Data with rich Python types
order = {
    "id": 12345,
    "customer": "John Doe",
    "total": Decimal("199.99"),
    "order_date": date(2025, 12, 4),
    "shipped": False,
}

print("Original data:")
for k, v in order.items():
    print(f"  {k}: {v} ({type(v).__name__})")

In [ ]:
# With standard JSON, it doesn't work!
try:
    json.dumps(order)
except TypeError as e:
    print(f"Standard JSON error: {e}")

## Solution 1: `as_json()` - Standard JSON

Converts Python types to readable JSON format, but **loses type information**.

In [ ]:
json_standard = as_json(order)
print("Standard JSON:")
print(json_standard)

In [ ]:
# If we re-read it, we lose the types!
parsed = json.loads(json_standard)
print("\nAfter parsing:")
for k, v in parsed.items():
    print(f"  {k}: {v} ({type(v).__name__})")

print("\nWarning: Decimal became float, date became string!")

## Solution 2: `as_typed_json()` - JSON with TYTX Types

Preserves type information using the `::CODE` notation.

In [ ]:
json_typed = as_typed_json(order)
print("Typed JSON (TYTX):")
print(json_typed)

In [ ]:
# Note the type suffixes:
# - 12345::L      -> int
# - 199.99::N     -> Decimal
# - 2025-12-04::D -> date
# - false::B      -> bool

## `from_json()` - Parsing with Hydration

Automatically reconstructs the original Python types.

In [ ]:
# Parsing typed JSON
restored = from_json(json_typed)

print("Restored data:")
for k, v in restored.items():
    print(f"  {k}: {v} ({type(v).__name__})")

print("\nTypes preserved correctly!")

In [ ]:
# Verify equality
print("Original == Restored?", order == restored)

## Nested Structures

TYTX correctly handles nested objects and lists.

In [ ]:
invoice = {
    "number": "INV-2025-001",
    "date": date(2025, 12, 4),
    "customer": {
        "name": "Acme Corp",
        "vat_id": "US123456789",
    },
    "lines": [
        {"description": "Product A", "quantity": 2, "price": Decimal("50.00")},
        {"description": "Product B", "quantity": 1, "price": Decimal("99.99")},
    ],
    "total": Decimal("199.99"),
    "paid": False,
}

json_invoice = as_typed_json(invoice, indent=2)
print(json_invoice)

In [ ]:
# Roundtrip
restored_invoice = from_json(json_invoice)

print("Verify lines:")
for line in restored_invoice["lines"]:
    print(f"  {line['description']}: {line['price']} ({type(line['price']).__name__})")

## DateTime with Timezone

In [ ]:
event = {
    "title": "Meeting",
    "start": datetime(2025, 12, 4, 14, 30, tzinfo=timezone.utc),
    "duration_minutes": 60,
}

json_event = as_typed_json(event)
print(json_event)

In [ ]:
restored_event = from_json(json_event)
print(f"Start: {restored_event['start']}")
print(f"Type: {type(restored_event['start']).__name__}")
print(f"Timezone: {restored_event['start'].tzinfo}")

## Comparison: Standard JSON vs TYTX

In [ ]:
data = {
    "price": Decimal("29.99"),
    "quantity": 5,
    "date": date(2025, 1, 15),
}

print("=" * 50)
print("STANDARD JSON (as_json):")
print(as_json(data))
print()
print("TYTX JSON (as_typed_json):")
print(as_typed_json(data))
print("=" * 50)

## When to Use What?

| Function | When to Use |
|----------|-------------|
| `as_json()` | APIs to external systems that don't support TYTX |
| `as_typed_json()` | Communication between systems that support TYTX |
| `from_json()` | JSON parsing (auto-detects if it's TYTX) |

## Automatic Detection

`from_json()` automatically detects if JSON contains TYTX values.

In [ ]:
from genro_tytx import is_tytx_payload, detect_tytx_mode

normal_json = '{"name": "test", "value": 42}'
tytx_json = '{"name": "test", "value": "42::L"}'

print(f"Normal JSON - is_tytx: {is_tytx_payload(normal_json)}")
print(f"TYTX JSON   - is_tytx: {is_tytx_payload(tytx_json)}")

In [ ]:
# from_json works with both
print("From normal JSON:", from_json(normal_json))
print("From TYTX JSON:", from_json(tytx_json))

## Next Notebook

In the next notebook we'll explore **Structs**: structured schemas for defining complex types with metadata.